In [ ]:
import bs4 # TODO: Achar o motto do bs4
from bs4.element import NavigableString
import re, logging
import dateutil.parser
import pandas as pd
from collections import namedtuple
import settings

logging.basicConfig(level=logging.WARNING)
log = logging.getLogger()

In [ ]:
Msg = namedtuple('Msg', 'text name datetime quote')
SubjectChange = namedtuple('SubjectChange', 'subject phone datetime')

In [ ]:
with open(f'{settings.GROUP_NAME}.html') as f:
#with open(f'sample.html') as f:
    html = f.read()
    soup = bs4.BeautifulSoup(html, 'html5lib')

In [ ]:
def get_messages(soup): # TODO: Emojis, Images, Quotes, Audios
    msgs, subject_changes = [], []
    date, phone = None, None
    global DEBUG
    body = soup.find('body')
    for div in body.select('[data-pre-plain-text]'):
        log.debug('------------------')
        log.debug(div)
        datetime_author = div.attrs['data-pre-plain-text']
        match = re.search(r'\[(.*)\] (.*): ', div.attrs['data-pre-plain-text'])
        datetime, author = match.groups()
        datetime = dateutil.parser.parse(datetime)
        quote = div.select('span.quoted-mention')
        if quote:
            quote = quote[0].parent.parent
            #DEBUG = quote
            quoted_person = quote.find('span').text
            quote.decompose()
        else:
            quoted_person = None
        DEBUG = div
        text = div.select('span.selectable-text')
        if text:
            for i in text[0].select('img.emoji'):
                i.replace_with(i.attrs['alt'])
            text = text[0].text
        else:
            text = div.select('div.selectable-text img')[0].attrs['alt']
        msg = Msg(text, author, datetime, quoted_person)
        log.info(f'Got a msg {msg}')
        msgs.append(msg)  
    return msgs


messages = get_messages(soup)
messages[:20] # TODO: Resolver quotes q estao vazando no texto dos outros

In [ ]:
%%script echo skip
a = DEBUG.select('span.selectable-text')
b = a[0].select('img.emoji')[0]
t = b.attrs['alt']
b.replace_with(t)


In [ ]:
df = pd.DataFrame(messages)
df.to_csv(f'{settings.GROUP_NAME}_messages.csv', index=False)

In [ ]:
df.quote.value_counts()

In [ ]:
df.text.iloc[1]

In [ ]:
df